#SVR FOR AM-I

In [1]:
import os
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import KFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ========== Configuration ==========
DATA_FOLDER = './train_test_split'
OUTPUT_FOLDER = './svr-models'
MODEL_SAVE_FOLDER = os.path.join(OUTPUT_FOLDER, 'AM-I-svr-model')
SEED = 42

FEATURE_COLS = ['MolWt', 'logP', 'TPSA', 'H_bond_donors', 'H_bond_acceptors']
FP_COLS = [f'col{i}' for i in range(823)]
MG_COLS = [f'fp_{i}' for i in range(1024)]
ALL_FEATURES = FEATURE_COLS + FP_COLS + MG_COLS
TARGET_COL = 'UV_RT-s'

os.makedirs(MODEL_SAVE_FOLDER, exist_ok=True)

# Global evaluation results
all_eval_results = []

# iPhone color scheme (fresh style)
IPHONE_COLORS = {
    "scatter": "#007AFF",
    "line": "#FF3B30",
    "residual": "#34C759",
    "text": "#1C1C1E"
}

def load_and_prepare_data(train_file, test_file):
    """Load and preprocess training and testing data."""
    train_df = pd.read_csv(train_file).dropna(subset=ALL_FEATURES + [TARGET_COL])
    test_df = pd.read_csv(test_file).dropna(subset=ALL_FEATURES + [TARGET_COL])

    X_train = train_df[ALL_FEATURES].values
    y_train = train_df[TARGET_COL].values
    X_test = test_df[ALL_FEATURES].values
    y_test = test_df[TARGET_COL].values

    # Standardize feature columns only
    scaler = StandardScaler()
    X_train[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train[:, :len(FEATURE_COLS)])
    X_test[:, :len(FEATURE_COLS)] = scaler.transform(X_test[:, :len(FEATURE_COLS)])

    return X_train, y_train, X_test, y_test, scaler

def objective(trial, X, y):
    """Optuna objective function for hyperparameter tuning."""
    C = trial.suggest_float('C', 1e-2, 1e3, log=True)
    gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-3, 1.0, log=True)

    model = SVR(C=C, gamma=gamma, epsilon=epsilon)
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx].copy(), X[val_idx].copy()
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Scale features for each fold
        scaler = StandardScaler()
        X_train_fold[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train_fold[:, :len(FEATURE_COLS)])
        X_val_fold[:, :len(FEATURE_COLS)] = scaler.transform(X_val_fold[:, :len(FEATURE_COLS)])

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        scores.append(r2_score(y_val_fold, y_pred))

    return np.mean(scores)

def plot_learning_curve(estimator, X, y, title, save_path):
    """Generate and save learning curve plot."""
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, cv=5, scoring='r2', train_sizes=np.linspace(0.1, 1.0, 5), random_state=SEED)

    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, valid_scores_mean, label='Validation score')
    plt.xlabel("Training Set Size")
    plt.ylabel("R² Score")
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_scatter_and_residuals(y_true, y_pred, base_name):
    """Generate scatter plot and residual plot."""
    
    # Scatter plot
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_true, y_pred, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)],
             linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    plt.xlabel("True Retention Time (s)")
    plt.ylabel("Predicted Retention Time (s)")
    plt.title("")
    plt.text(0.5, -0.15, "Predicted vs True Values", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", 
             transform=ax.transAxes, verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scatter.png"))
    plt.close()

    # Residual plot
    residuals = y_pred - y_true
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_pred, residuals, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.axhline(y=0, linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    plt.xlabel("Predicted Retention Time (s)")
    plt.ylabel("Residuals (Predicted - True)")
    plt.title("")
    plt.text(0.5, -0.15, "Residual Plot", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", 
             transform=ax.transAxes, verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_residuals.png"))
    plt.close()

def train_and_evaluate(train_csv, test_csv):
    """Main training and evaluation pipeline."""
    base_name = os.path.splitext(os.path.basename(train_csv))[0].replace("_train", "")
    print(f"\n🚀 Training on dataset: {base_name}")

    X_train, y_train, X_test, y_test, scaler = load_and_prepare_data(train_csv, test_csv)

    # Hyperparameter optimization
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=30)

    best_params = study.best_params
    model = SVR(**best_params)
    model.fit(X_train, y_train)

    # Model evaluation
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    print(f"📊 R²: {r2:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

    # Save model and scaler
    joblib.dump(model, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_svr_model.joblib"))
    joblib.dump(scaler, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scaler.joblib"))

    # Save predictions
    pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).to_csv(
        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_predictions.csv"), index=False
    )

    # Generate plots
    plot_learning_curve(SVR(**best_params), X_train, y_train,
                        f"Learning Curve - {base_name}",
                        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_learning_curve.png"))

    plot_scatter_and_residuals(y_test, y_pred, base_name)

    # Record results
    all_eval_results.append({
        "Dataset": base_name,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "C": best_params['C'],
        "gamma": best_params['gamma'],
        "epsilon": best_params['epsilon']
    })

    # Save summary
    summary_df = pd.DataFrame(all_eval_results)
    summary_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, "AM-I-svr_model_evaluation_summary.csv"), index=False)
    print(f"\n✅ All model evaluation results saved to: AM-I-svr_model_evaluation_summary.csv")

if __name__ == "__main__":
    train_csv = os.path.join(DATA_FOLDER, "AM-I-filtered_with_labels_k4_train.csv")
    test_csv = os.path.join(DATA_FOLDER, "AM-I-filtered_with_labels_k4_test.csv")
    train_and_evaluate(train_csv, test_csv)

/home/xuxianyan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🚀 Training on dataset: AM-I-filtered_with_labels_k4


[I 2026-01-18 10:53:20,860] A new study created in memory with name: no-name-b4592c46-6605-40ff-8303-aa502104f31d
[I 2026-01-18 10:55:38,100] Trial 0 finished with value: -0.003930982826524598 and parameters: {'C': 0.7459343285726545, 'gamma': 5.669849511478847, 'epsilon': 0.15702970884055384}. Best is trial 0 with value: -0.003930982826524598.
[I 2026-01-18 10:57:54,376] Trial 1 finished with value: 0.6184433368873307 and parameters: {'C': 9.846738873614559, 'gamma': 0.0006026889128682511, 'epsilon': 0.0029375384576328283}. Best is trial 1 with value: 0.6184433368873307.
[I 2026-01-18 11:00:11,877] Trial 2 finished with value: -0.009444576488156508 and parameters: {'C': 0.0195172246414495, 'gamma': 2.1423021757741068, 'epsilon': 0.06358358856676251}. Best is trial 1 with value: 0.6184433368873307.
[I 2026-01-18 11:02:16,831] Trial 3 finished with value: 0.5856733757479251 and parameters: {'C': 34.70266988650411, 'gamma': 0.00012674255898937226, 'epsilon': 0.8123245085588685}. Best is 

📊 R²: 0.9145 | RMSE: 3.7973 | MAE: 2.8457

✅ All model evaluation results saved to: AM-I-svr_model_evaluation_summary.csv


#svr for AM-II

In [3]:
import os
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import KFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ========== Configuration ==========
DATA_FOLDER = './train_test_split'
OUTPUT_FOLDER = './svr-models'
MODEL_SAVE_FOLDER = os.path.join(OUTPUT_FOLDER, 'AM-II-svr-model')
SEED = 42

FEATURE_COLS = ['MolWt', 'logP', 'TPSA', 'H_bond_donors', 'H_bond_acceptors']
FP_COLS = [f'col{i}' for i in range(823)]
MG_COLS = [f'fp_{i}' for i in range(1024)]
ALL_FEATURES = FEATURE_COLS + FP_COLS + MG_COLS
TARGET_COL = 'UV_RT-s'

os.makedirs(MODEL_SAVE_FOLDER, exist_ok=True)

# Global evaluation results
all_eval_results = []

# iPhone color palette (clean style)
IPHONE_COLORS = {
    "scatter": "#007AFF",
    "line": "#FF3B30",
    "residual": "#34C759",
    "text": "#1C1C1E"
}

def load_and_prepare_data(train_file, test_file):
    """Load and preprocess training and test data."""
    train_df = pd.read_csv(train_file).dropna(subset=ALL_FEATURES + [TARGET_COL])
    test_df = pd.read_csv(test_file).dropna(subset=ALL_FEATURES + [TARGET_COL])

    X_train = train_df[ALL_FEATURES].values
    y_train = train_df[TARGET_COL].values
    X_test = test_df[ALL_FEATURES].values
    y_test = test_df[TARGET_COL].values

    scaler = StandardScaler()
    X_train[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train[:, :len(FEATURE_COLS)])
    X_test[:, :len(FEATURE_COLS)] = scaler.transform(X_test[:, :len(FEATURE_COLS)])

    return X_train, y_train, X_test, y_test, scaler

def objective(trial, X, y):
    """Optuna objective function for hyperparameter optimization."""
    C = trial.suggest_float('C', 1e-2, 1e3, log=True)
    gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-3, 1.0, log=True)

    model = SVR(C=C, gamma=gamma, epsilon=epsilon)
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx].copy(), X[val_idx].copy()
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_train_fold[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train_fold[:, :len(FEATURE_COLS)])
        X_val_fold[:, :len(FEATURE_COLS)] = scaler.transform(X_val_fold[:, :len(FEATURE_COLS)])

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        scores.append(r2_score(y_val_fold, y_pred))

    return np.mean(scores)

def plot_learning_curve(estimator, X, y, title, save_path):
    """Plot learning curve for model evaluation."""
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, cv=5, scoring='r2', train_sizes=np.linspace(0.1, 1.0, 5), random_state=SEED)

    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, valid_scores_mean, label='Validation score')
    plt.xlabel("Training Set Size")
    plt.ylabel("R2 Score")
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_scatter_and_residuals(y_true, y_pred, base_name):
    """Generate scatter and residual plots."""
    # Scatter plot
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_true, y_pred, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)],
             linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    plt.xlabel("True Retention Time (s)")
    plt.ylabel("Predicted Retention Time (s)")
    plt.title("")
    plt.text(0.5, -0.15, "Predicted vs True Values", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", transform=ax.transAxes, 
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scatter.png"))
    plt.close()

    # Residual plot
    residuals = y_pred - y_true
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_pred, residuals, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.axhline(y=0, linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2_res = r2_score(y_true, y_pred)
    mae_res = mean_absolute_error(y_true, y_pred)

    plt.xlabel("Predicted Retention Time (s)")
    plt.ylabel("Residuals (Predicted - True)")
    plt.title("")
    plt.text(0.5, -0.15, "Residual Plot", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2_res:.3f}\nMAE = {mae_res:.3f}", transform=ax.transAxes, 
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_residuals.png"))
    plt.close()

def train_and_evaluate(train_csv, test_csv):
    """Main training and evaluation pipeline."""
    base_name = os.path.splitext(os.path.basename(train_csv))[0].replace("_train", "")
    print(f"\n🚀 Training on dataset: {base_name}")

    X_train, y_train, X_test, y_test, scaler = load_and_prepare_data(train_csv, test_csv)

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=30)

    best_params = study.best_params
    model = SVR(**best_params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    print(f"📊 R2: {r2:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

    joblib.dump(model, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_svr_model.joblib"))
    joblib.dump(scaler, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scaler.joblib"))

    pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).to_csv(
        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_predictions.csv"), index=False
    )

    plot_learning_curve(SVR(**best_params), X_train, y_train,
                        f"Learning Curve - {base_name}",
                        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_learning_curve.png"))

    plot_scatter_and_residuals(y_test, y_pred, base_name)

    all_eval_results.append({
        "Dataset": base_name,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "C": best_params['C'],
        "gamma": best_params['gamma'],
        "epsilon": best_params['epsilon']
    })

    summary_df = pd.DataFrame(all_eval_results)
    summary_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, "AM-II-Neutral-svr_model_evaluation_summary.csv"), index=False)
    print(f"\n✅ All model evaluation results saved to: AM-II-Neutral-svr_model_evaluation_summary.csv")

if __name__ == "__main__":
    train_csv = os.path.join(DATA_FOLDER, "AM-II-filtered_with_labels_k3_train.csv")
    test_csv = os.path.join(DATA_FOLDER, "AM-II-filtered_with_labels_k3_test.csv")
    train_and_evaluate(train_csv, test_csv)


🚀 Training on dataset: AM-II-filtered_with_labels_k3


[I 2026-01-18 12:12:58,315] A new study created in memory with name: no-name-e984788d-f078-4689-9eb7-7f60abc8ffc6
[I 2026-01-18 12:13:04,288] Trial 0 finished with value: -0.027286032614959987 and parameters: {'C': 0.7459343285726545, 'gamma': 5.669849511478847, 'epsilon': 0.15702970884055384}. Best is trial 0 with value: -0.027286032614959987.
[I 2026-01-18 12:13:10,383] Trial 1 finished with value: 0.5247742466641797 and parameters: {'C': 9.846738873614559, 'gamma': 0.0006026889128682511, 'epsilon': 0.0029375384576328283}. Best is trial 1 with value: 0.5247742466641797.
[I 2026-01-18 12:13:16,420] Trial 2 finished with value: -0.025673500376314794 and parameters: {'C': 0.0195172246414495, 'gamma': 2.1423021757741068, 'epsilon': 0.06358358856676251}. Best is trial 1 with value: 0.5247742466641797.
[I 2026-01-18 12:13:21,613] Trial 3 finished with value: 0.4792961381370079 and parameters: {'C': 34.70266988650411, 'gamma': 0.00012674255898937226, 'epsilon': 0.8123245085588685}. Best is 

📊 R2: 0.8917 | RMSE: 3.0097 | MAE: 2.0730

✅ All model evaluation results saved to: AM-II-Neutral-svr_model_evaluation_summary.csv


#svr for AM-III

In [2]:
import os
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from sklearn.model_selection import KFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ========== Configuration ==========
DATA_FOLDER = './train_test_split'
OUTPUT_FOLDER = './svr-models'
MODEL_SAVE_FOLDER = os.path.join(OUTPUT_FOLDER, 'AM-III-svr-model')
SEED = 42

FEATURE_COLS = ['MolWt', 'logP', 'TPSA', 'H_bond_donors', 'H_bond_acceptors']
FP_COLS = [f'col{i}' for i in range(823)]
MG_COLS = [f'fp_{i}' for i in range(1024)]
ALL_FEATURES = FEATURE_COLS + FP_COLS + MG_COLS
TARGET_COL = 'UV_RT-s'

os.makedirs(MODEL_SAVE_FOLDER, exist_ok=True)

# Global evaluation results
all_eval_results = []

# iPhone color palette (clean style)
IPHONE_COLORS = {
    "scatter": "#007AFF",
    "line": "#FF3B30",
    "residual": "#34C759",
    "text": "#1C1C1E"
}

def load_and_prepare_data(train_file, test_file):
    """Load and preprocess training and test data."""
    train_df = pd.read_csv(train_file).dropna(subset=ALL_FEATURES + [TARGET_COL])
    test_df = pd.read_csv(test_file).dropna(subset=ALL_FEATURES + [TARGET_COL])

    X_train = train_df[ALL_FEATURES].values
    y_train = train_df[TARGET_COL].values
    X_test = test_df[ALL_FEATURES].values
    y_test = test_df[TARGET_COL].values

    scaler = StandardScaler()
    X_train[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train[:, :len(FEATURE_COLS)])
    X_test[:, :len(FEATURE_COLS)] = scaler.transform(X_test[:, :len(FEATURE_COLS)])

    return X_train, y_train, X_test, y_test, scaler

def objective(trial, X, y):
    """Optuna objective function for hyperparameter optimization."""
    C = trial.suggest_float('C', 1e-2, 1e3, log=True)
    gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True)
    epsilon = trial.suggest_float('epsilon', 1e-3, 1.0, log=True)

    model = SVR(C=C, gamma=gamma, epsilon=epsilon)
    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_fold, X_val_fold = X[train_idx].copy(), X[val_idx].copy()
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        scaler = StandardScaler()
        X_train_fold[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train_fold[:, :len(FEATURE_COLS)])
        X_val_fold[:, :len(FEATURE_COLS)] = scaler.transform(X_val_fold[:, :len(FEATURE_COLS)])

        model.fit(X_train_fold, y_train_fold)
        y_pred = model.predict(X_val_fold)
        scores.append(r2_score(y_val_fold, y_pred))

    return np.mean(scores)

def plot_learning_curve(estimator, X, y, title, save_path):
    """Plot learning curve for model evaluation."""
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, cv=5, scoring='r2', train_sizes=np.linspace(0.1, 1.0, 5), random_state=SEED)

    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, valid_scores_mean, label='Validation score')
    plt.xlabel("Training Set Size")
    plt.ylabel("R2 Score")
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_scatter_and_residuals(y_true, y_pred, base_name):
    """Generate scatter and residual plots."""
    # Scatter plot
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_true, y_pred, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)],
             linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2 = r2_score(y_true, y_pred)
    mae = mean_absolute_error(y_true, y_pred)

    plt.xlabel("True Retention Time (s)")
    plt.ylabel("Predicted Retention Time (s)")
    plt.title("")
    plt.text(0.5, -0.15, "Predicted vs True Values", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", transform=ax.transAxes, 
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scatter.png"))
    plt.close()

    # Residual plot
    residuals = y_pred - y_true
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    ax.spines['top'].set_visible(True)
    ax.spines['right'].set_visible(True)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    plt.grid(False)

    plt.scatter(y_pred, residuals, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.axhline(y=0, linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2_res = r2_score(y_true, y_pred)
    mae_res = mean_absolute_error(y_true, y_pred)

    plt.xlabel("Predicted Retention Time (s)")
    plt.ylabel("Residuals (Predicted - True)")
    plt.title("")
    plt.text(0.5, -0.15, "Residual Plot", ha='center', va='center', 
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2_res:.3f}\nMAE = {mae_res:.3f}", transform=ax.transAxes, 
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_residuals.png"))
    plt.close()

def train_and_evaluate(train_csv, test_csv):
    """Main training and evaluation pipeline."""
    base_name = os.path.splitext(os.path.basename(train_csv))[0].replace("_train", "")
    print(f"\n🚀 Training on dataset: {base_name}")

    X_train, y_train, X_test, y_test, scaler = load_and_prepare_data(train_csv, test_csv)

    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=SEED))
    study.optimize(lambda trial: objective(trial, X_train, y_train), n_trials=30)

    best_params = study.best_params
    model = SVR(**best_params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)

    print(f"📊 R2: {r2:.4f} | RMSE: {rmse:.4f} | MAE: {mae:.4f}")

    joblib.dump(model, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_svr_model.joblib"))
    joblib.dump(scaler, os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_scaler.joblib"))

    pd.DataFrame({'y_true': y_test, 'y_pred': y_pred}).to_csv(
        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_predictions.csv"), index=False
    )

    plot_learning_curve(SVR(**best_params), X_train, y_train,
                        f"Learning Curve - {base_name}",
                        os.path.join(MODEL_SAVE_FOLDER, f"{base_name}_learning_curve.png"))

    plot_scatter_and_residuals(y_test, y_pred, base_name)

    all_eval_results.append({
        "Dataset": base_name,
        "R2": r2,
        "RMSE": rmse,
        "MAE": mae,
        "C": best_params['C'],
        "gamma": best_params['gamma'],
        "epsilon": best_params['epsilon']
    })

    summary_df = pd.DataFrame(all_eval_results)
    summary_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, "AM-III-svr_model_evaluation_summary.csv"), index=False)
    print(f"\n✅ All model evaluation results saved to: AM-III-svr_model_evaluation_summary.csv")

if __name__ == "__main__":
    train_csv = os.path.join(DATA_FOLDER, "AM-III-filtered_with_labels_k4_train.csv")
    test_csv = os.path.join(DATA_FOLDER, "AM-III-filtered_with_labels_k4_test.csv")
    train_and_evaluate(train_csv, test_csv)


🚀 Training on dataset: AM-III-filtered_with_labels_k4


[I 2026-01-18 12:11:19,025] A new study created in memory with name: no-name-e3b730e0-6e32-48f5-a118-b37d4e51d43a
[I 2026-01-18 12:11:21,777] Trial 0 finished with value: -0.016026618949243555 and parameters: {'C': 0.7459343285726545, 'gamma': 5.669849511478847, 'epsilon': 0.15702970884055384}. Best is trial 0 with value: -0.016026618949243555.
[I 2026-01-18 12:11:24,523] Trial 1 finished with value: 0.5435040417541912 and parameters: {'C': 9.846738873614559, 'gamma': 0.0006026889128682511, 'epsilon': 0.0029375384576328283}. Best is trial 1 with value: 0.5435040417541912.
[I 2026-01-18 12:11:27,301] Trial 2 finished with value: -0.01716512868399751 and parameters: {'C': 0.0195172246414495, 'gamma': 2.1423021757741068, 'epsilon': 0.06358358856676251}. Best is trial 1 with value: 0.5435040417541912.
[I 2026-01-18 12:11:29,849] Trial 3 finished with value: 0.5004563865397835 and parameters: {'C': 34.70266988650411, 'gamma': 0.00012674255898937226, 'epsilon': 0.8123245085588685}. Best is t

📊 R2: 0.8775 | RMSE: 5.1064 | MAE: 3.2658

✅ All model evaluation results saved to: AM-III-svr_model_evaluation_summary.csv


#svr for AM-IV, AM-V, AM-VI, AM-VII with nestcv

In [4]:
import os
import glob
import joblib
import optuna
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.svm import SVR
from sklearn.model_selection import KFold, learning_curve
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings

warnings.filterwarnings("ignore")

# ----------------- Configuration -----------------
DATA_FOLDER = './processed_results'                 # Data folder (CSV)
MODEL_SAVE_FOLDER = './svr-model-other4'
SEED = 42
np.random.seed(SEED)

# Keep consistent feature columns
FEATURE_COLS = ['MolWt', 'logP', 'TPSA', 'H_bond_donors', 'H_bond_acceptors']
FP_COLS = [f'col{i}' for i in range(823)]
MG_COLS = [f'fp_{i}' for i in range(1024)]
ALL_FEATURES = FEATURE_COLS + FP_COLS + MG_COLS
TARGET_COL = 'UV_RT-s'

# Allowed file name prefixes (user specified)
ALLOWED_PREFIXES = {
    "AM-IV-filtered",
    "AM-V-filtered", 
    "AM-VI-filtered",
    "AM-VII-filtered"
}

# Use 26 cores
N_JOBS = 26

os.makedirs(MODEL_SAVE_FOLDER, exist_ok=True)

IPHONE_COLORS = {
    "scatter": "#007AFF",
    "line": "#FF3B30",
    "residual": "#34C759",
    "text": "#1C1C1E"
}

# ----------------- Data Loading -----------------
def load_data():
    data_files = glob.glob(os.path.join(DATA_FOLDER, '*.csv'))
    dfs = []
    for f in data_files:
        file_prefix = os.path.splitext(os.path.basename(f))[0]
        if file_prefix not in ALLOWED_PREFIXES:
            continue
        df = pd.read_csv(f)
        needed_cols = set(ALL_FEATURES + [TARGET_COL])
        if not needed_cols.issubset(set(df.columns)):
            print(f"Warning: file {f} missing required columns, skipping.")
            continue
        df = df.dropna(subset=ALL_FEATURES + [TARGET_COL]).copy()
        if df.shape[0] == 0:
            print(f"Warning: file {f} has no valid rows after dropna, skipping.")
            continue
        df['file_prefix'] = file_prefix
        dfs.append(df)
    if len(dfs) == 0:
        raise RuntimeError("No valid data files found for the allowed prefixes.")
    data = pd.concat(dfs, ignore_index=True)
    return data

# ----------------- Plotting Helpers -----------------
def plot_learning_curve(estimator, X, y, title, save_path):
    train_sizes, train_scores, valid_scores = learning_curve(
        estimator, X, y, cv=5, scoring='r2',
        train_sizes=np.linspace(0.1, 1.0, 5), random_state=SEED, n_jobs=N_JOBS)

    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)

    plt.figure()
    plt.plot(train_sizes, train_scores_mean, label='Training score')
    plt.plot(train_sizes, valid_scores_mean, label='Validation score')
    plt.xlabel("Training Set Size")
    plt.ylabel("R2 Score")
    plt.title(title)
    plt.legend(loc='best')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_scatter_and_residuals(y_true, y_pred, save_prefix):
    # Scatter plot
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    for spine in ['top', 'right', 'bottom', 'left']:
        ax.spines[spine].set_visible(True)
    plt.grid(False)

    plt.scatter(y_true, y_pred, alpha=0.6, color=IPHONE_COLORS['scatter'])
    if len(y_true) > 0:
        plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)],
                 linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    r2 = r2_score(y_true, y_pred) if len(y_true) > 0 else np.nan
    mae = mean_absolute_error(y_true, y_pred) if len(y_true) > 0 else np.nan

    plt.xlabel("True Retention Time (s)")
    plt.ylabel("Predicted Retention Time (s)")
    plt.text(0.5, -0.15, "Predicted vs True Values", ha='center', va='center',
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", transform=ax.transAxes,
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_scatter.png")
    plt.close()

    # Residual plot
    residuals = y_pred - y_true
    plt.figure(figsize=(6, 6))
    ax = plt.gca()
    ax.tick_params(axis='both', direction='out', length=6, width=1.2)
    for spine in ['top', 'right', 'bottom', 'left']:
        ax.spines[spine].set_visible(True)
    plt.grid(False)

    plt.scatter(y_pred, residuals, alpha=0.6, color=IPHONE_COLORS['scatter'])
    plt.axhline(y=0, linestyle='--', color=IPHONE_COLORS['line'], linewidth=1)

    plt.xlabel("Predicted Retention Time (s)")
    plt.ylabel("Residuals (Predicted - True)")
    plt.text(0.5, -0.15, "Residual Plot", ha='center', va='center',
             transform=ax.transAxes, fontsize=12, color=IPHONE_COLORS['text'])
    plt.text(0.05, 0.95, f"R² = {r2:.3f}\nMAE = {mae:.3f}", transform=ax.transAxes,
             verticalalignment='top', fontsize=10, color=IPHONE_COLORS['text'])
    plt.tight_layout()
    plt.savefig(f"{save_prefix}_residuals.png")
    plt.close()

# ----------------- Optuna Objective -----------------
def make_inner_objective(X_train, y_train, n_inner_splits=3):
    def objective(trial):
        C = trial.suggest_float('C', 1e-2, 1e3, log=True)
        gamma = trial.suggest_float('gamma', 1e-4, 1e1, log=True)
        epsilon = trial.suggest_float('epsilon', 1e-3, 1.0, log=True)

        model = SVR(C=C, gamma=gamma, epsilon=epsilon)
        kf_inner = KFold(n_splits=n_inner_splits, shuffle=True, random_state=SEED)

        inner_scores = []
        for tr_idx, val_idx in kf_inner.split(X_train):
            X_tr, X_val = X_train[tr_idx].copy(), X_train[val_idx].copy()
            y_tr, y_val = y_train[tr_idx], y_train[val_idx]

            scaler = StandardScaler()
            X_tr[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_tr[:, :len(FEATURE_COLS)])
            X_val[:, :len(FEATURE_COLS)] = scaler.transform(X_val[:, :len(FEATURE_COLS)])

            model.fit(X_tr, y_tr)
            y_pred = model.predict(X_val)
            inner_scores.append(r2_score(y_val, y_pred))

        return np.mean(inner_scores)
    return objective

# ----------------- Nested CV -----------------
def nested_cv_evaluate(X, y, outer_splits=5, inner_splits=3, n_trials=100):
    kf_outer = KFold(n_splits=outer_splits, shuffle=True, random_state=SEED)
    fold_results = []

    for fold_idx, (train_idx, val_idx) in enumerate(kf_outer.split(X), 1):
        print(f"\n--- Outer Fold {fold_idx}/{outer_splits} ---")
        X_train, X_val = X[train_idx].copy(), X[val_idx].copy()
        y_train, y_val = y[train_idx], y[val_idx]

        study = optuna.create_study(direction='maximize',
                                    sampler=optuna.samplers.TPESampler(seed=SEED))
        objective = make_inner_objective(X_train, y_train, n_inner_splits=inner_splits)
        study.optimize(objective, n_trials=n_trials, n_jobs=N_JOBS)
        best_params = study.best_params
        best_value = study.best_value
        print(f"  Inner best params: {best_params}, inner CV mean R2 = {best_value:.4f}")

        scaler = StandardScaler()
        X_train[:, :len(FEATURE_COLS)] = scaler.fit_transform(X_train[:, :len(FEATURE_COLS)])
        X_val[:, :len(FEATURE_COLS)] = scaler.transform(X_val[:, :len(FEATURE_COLS)])

        model = SVR(**best_params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        r2 = r2_score(y_val, y_pred)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        mae = mean_absolute_error(y_val, y_pred)

        print(f"  Outer fold {fold_idx} metrics - R2: {r2:.4f}, RMSE: {rmse:.4f}, MAE: {mae:.4f}")

        fold_results.append({
            'fold': fold_idx,
            'best_params': best_params,
            'inner_best_value': best_value,
            'r2': r2,
            'rmse': rmse,
            'mae': mae,
            'y_true': y_val,
            'y_pred': y_pred
        })

    r2s = [f['r2'] for f in fold_results]
    rmses = [f['rmse'] for f in fold_results]
    maes = [f['mae'] for f in fold_results]

    summary = {
        'r2_mean': np.mean(r2s),
        'r2_std': np.std(r2s, ddof=1),
        'rmse_mean': np.mean(rmses),
        'rmse_std': np.std(rmses, ddof=1),
        'mae_mean': np.mean(maes),
        'mae_std': np.std(maes, ddof=1)
    }

    return fold_results, summary

# ----------------- Single File Processing -----------------
def process_single_file(df, file_prefix,
                        outer_splits=5, inner_splits=3, n_trials=100):
    print(f"\n========== Processing {file_prefix} ==========")
    X = df[ALL_FEATURES].values
    y = df[TARGET_COL].values

    fold_results, summary = nested_cv_evaluate(X, y,
                                               outer_splits=outer_splits,
                                               inner_splits=inner_splits,
                                               n_trials=n_trials)

    print(f"\n{file_prefix} - Nested CV summary:")
    print(f"  R2: {summary['r2_mean']:.4f} ± {summary['r2_std']:.4f}")
    print(f"  RMSE: {summary['rmse_mean']:.4f} ± {summary['rmse_std']:.4f}")
    print(f"  MAE: {summary['mae_mean']:.4f} ± {summary['mae_std']:.4f}")

    # Save results
    fold_preds = []
    for fr in fold_results:
        fold_df = pd.DataFrame({
            'y_true': fr['y_true'],
            'y_pred': fr['y_pred']
        })
        fold_df['fold'] = fr['fold']
        fold_preds.append(fold_df)
    all_fold_preds_df = pd.concat(fold_preds, ignore_index=True)
    all_fold_preds_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_nestedcv_outer_preds.csv"),
                             index=False)

    summary_df = pd.DataFrame([{
        'file_prefix': file_prefix,
        'r2_mean': summary['r2_mean'],
        'r2_std': summary['r2_std'],
        'rmse_mean': summary['rmse_mean'],
        'rmse_std': summary['rmse_std'],
        'mae_mean': summary['mae_mean'],
        'mae_std': summary['mae_std']
    }])
    summary_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_nestedcv_summary.csv"), index=False)

    params_df = pd.DataFrame([fr['best_params'] for fr in fold_results])
    params_df.to_csv(os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_inner_best_params_per_fold.csv"), index=False)

    print("\nRunning final inner hyperparameter search on FULL data...")
    study_final = optuna.create_study(direction='maximize',
                                      sampler=optuna.samplers.TPESampler(seed=SEED))
    final_objective = make_inner_objective(X, y, n_inner_splits=inner_splits)
    study_final.optimize(final_objective, n_trials=n_trials, n_jobs=N_JOBS)
    final_best_params = study_final.best_params
    print(f"Final best params on FULL data: {final_best_params}")

    final_scaler = StandardScaler()
    X_scaled = X.copy()
    X_scaled[:, :len(FEATURE_COLS)] = final_scaler.fit_transform(X_scaled[:, :len(FEATURE_COLS)])
    final_model = SVR(**final_best_params)
    final_model.fit(X_scaled, y)

    model_path = os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_final_svr_model.joblib")
    scaler_path = os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_final_scaler.joblib")
    joblib.dump(final_model, model_path)
    joblib.dump(final_scaler, scaler_path)

    pd.DataFrame([final_best_params]).to_csv(
        os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_final_best_params.csv"), index=False)

    lc_path = os.path.join(MODEL_SAVE_FOLDER, f"{file_prefix}_learning_curve.png")
    plot_learning_curve(final_model, X_scaled, y, f"Learning Curve - {file_prefix}", lc_path)

    y_true_all = all_fold_preds_df['y_true'].values
    y_pred_all = all_fold_preds_df['y_pred'].values
    plot_prefix = os.path.join(MODEL_SAVE_FOLDER, file_prefix)
    plot_scatter_and_residuals(y_true_all, y_pred_all, plot_prefix)

    print(f"Saved final model to: {model_path}")
    print(f"Saved final scaler to: {scaler_path}")
    return summary

# ----------------- Main -----------------
def main():
    data = load_data()
    summaries = []
    for file_prefix, df_group in data.groupby('file_prefix'):
        s = process_single_file(df_group, file_prefix,
                                outer_splits=5, inner_splits=3, n_trials=100)
        summaries.append({'file_prefix': file_prefix, **s})
    pd.DataFrame(summaries).to_csv(os.path.join(MODEL_SAVE_FOLDER, "all_files_nestedcv_summary.csv"), index=False)

if __name__ == "__main__":
    main()

[I 2026-01-18 12:16:30,160] A new study created in memory with name: no-name-eb5c75b9-9c60-4c3e-8286-53f94fda50e7



========== Processing AM-IV-filtered ==========

--- Outer Fold 1/5 ---


[I 2026-01-18 12:16:30,602] Trial 1 finished with value: 0.7398295281035788 and parameters: {'C': 62.5238432689369, 'gamma': 0.0004884747714249566, 'epsilon': 0.7115145816863668}. Best is trial 1 with value: 0.7398295281035788.
[I 2026-01-18 12:16:30,704] Trial 2 finished with value: -0.032074570108723575 and parameters: {'C': 577.0624003828505, 'gamma': 5.643884317379479, 'epsilon': 0.0014667017513184187}. Best is trial 1 with value: 0.7398295281035788.
[I 2026-01-18 12:16:30,722] Trial 0 finished with value: 0.0629172824386045 and parameters: {'C': 7.0657455235022875, 'gamma': 0.0001303855851951483, 'epsilon': 0.0012882926986205175}. Best is trial 1 with value: 0.7398295281035788.
[I 2026-01-18 12:16:30,736] Trial 3 finished with value: -0.0195375008476798 and parameters: {'C': 0.2099666712412517, 'gamma': 0.000904375324258865, 'epsilon': 0.01600641083929421}. Best is trial 1 with value: 0.7398295281035788.
[I 2026-01-18 12:16:30,749] Trial 4 finished with value: 0.15587120023697398 

  Inner best params: {'C': 438.2092464763412, 'gamma': 0.0014790082230251454, 'epsilon': 0.001948256560601007}, inner CV mean R2 = 0.9133


[I 2026-01-18 12:16:35,474] A new study created in memory with name: no-name-8227bb54-a1f8-42a4-b726-6fc93274ea26


...

  Inner best params: {'C': 278.46640239086156, 'gamma': 0.010105693176327254, 'epsilon': 0.043120447702064524}, inner CV mean R2 = 0.9337
  Outer fold 4 metrics - R2: 0.9671, RMSE: 4.4445, MAE: 3.2301

--- Outer Fold 5/5 ---


[I 2026-01-18 12:18:21,788] Trial 0 finished with value: -0.040153542990083814 and parameters: {'C': 0.010312756522446738, 'gamma': 0.7402907051540272, 'epsilon': 0.004054832131388513}. Best is trial 0 with value: -0.040153542990083814.
[I 2026-01-18 12:18:21,920] Trial 3 finished with value: 0.2944379979958049 and parameters: {'C': 20.131555560990076, 'gamma': 0.17489725744061674, 'epsilon': 0.003263031572563642}. Best is trial 3 with value: 0.2944379979958049.
[I 2026-01-18 12:18:21,923] Trial 9 finished with value: -0.031453584254433974 and parameters: {'C': 0.019046473240579984, 'gamma': 0.004756967932646574, 'epsilon': 0.5914250391874974}. Best is trial 3 with value: 0.2944379979958049.
[I 2026-01-18 12:18:21,941] Trial 1 finished with value: -0.018846875046536666 and parameters: {'C': 27.42204163659823, 'gamma': 0.7388007054954147, 'epsilon': 0.023910028907836833}. Best is trial 3 with value: 0.2944379979958049.
[I 2026-01-18 12:18:21,952] Trial 5 finished with value: 0.064470175

  Inner best params: {'C': 85.84494366740265, 'gamma': 0.011614729374971511, 'epsilon': 0.006739335532246787}, inner CV mean R2 = 0.9612
  Outer fold 5 metrics - R2: 0.9428, RMSE: 5.1355, MAE: 3.5236

AM-VII-filtered - Nested CV summary:
  R2: 0.9389 ± 0.0311
  RMSE: 5.3348 ± 1.3730
  MAE: 3.5456 ± 0.5156

Running final inner hyperparameter search on FULL data...


[I 2026-01-18 12:18:27,201] Trial 4 finished with value: 0.9244722957212209 and parameters: {'C': 548.8066262050478, 'gamma': 0.0006453506167406127, 'epsilon': 0.018390166812208713}. Best is trial 4 with value: 0.9244722957212209.
[I 2026-01-18 12:18:27,268] Trial 8 finished with value: 0.9047189353732081 and parameters: {'C': 177.41653222861672, 'gamma': 0.00044433660958743894, 'epsilon': 0.9115712324480848}. Best is trial 4 with value: 0.9244722957212209.
[I 2026-01-18 12:18:27,287] Trial 3 finished with value: 0.25777654200770606 and parameters: {'C': 6.647757161302349, 'gamma': 0.000566520216832614, 'epsilon': 0.23934894620080885}. Best is trial 4 with value: 0.9244722957212209.
[I 2026-01-18 12:18:27,298] Trial 1 finished with value: 0.9273078947931731 and parameters: {'C': 62.582210318126506, 'gamma': 0.003954852859412689, 'epsilon': 0.0022307649394154513}. Best is trial 1 with value: 0.9273078947931731.
[I 2026-01-18 12:18:27,369] Trial 5 finished with value: 0.02759252789214268

Final best params on FULL data: {'C': 282.3467484838021, 'gamma': 0.009611163905791791, 'epsilon': 0.04216017634183765}
Saved final model to: ./svr-model-other4/AM-VII-filtered_final_svr_model.joblib
Saved final scaler to: ./svr-model-other4/AM-VII-filtered_final_scaler.joblib
